# Do some housekeeping on Stan's Excel worksheet

To do: 
1. Ensure ALL category names are consistent (no extra spaces, typos)
2. Spellcheck on all article titles
3. Display all author names for sanity check

In [15]:
import os
from openai import OpenAI
from tqdm import tqdm
import pandas as pd
from dotenv import load_dotenv

load_dotenv() #load your custom environment variables from .env file in same directory

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

## Load data

In [3]:
# Load the Excel file
excel_file = 'GRIT_INDEX_2024_10_OCT_backthrough2015.xlsx'

df_index = pd.read_excel(excel_file, sheet_name='INDEX', header=0)
df_histogram = pd.read_excel(excel_file, sheet_name='Histogram', header=2)

# Trim down relevant columns
df_index = df_index[df_index.columns[0:6]]

# Remove rows where there is not an entry in every column
df_index = df_index.dropna(how='any')
df_histogram = df_histogram.dropna(how='any')


# Convert all numbers to integers, then strings, and replace NaN values with empty strings
df_index = df_index.fillna('').map(lambda x: int(x) if isinstance(x, float) else x).astype('str')
df_histogram = df_histogram.fillna('').map(lambda x: int(x) if isinstance(x, float) else x).astype('str')



In [4]:
df_index

,Title,First Author,~Relevant Committee,TOPIC,Year,Month
0,SHHA Annual Meeting announcements,SHHA,SHHA,ANN MTG/REPORTS,2015,1
1,Covenant Violation,Dick Wavrik,CSC,COVENANTS GENERAL,2015,1
2,Bernalilloo County Ordinance Regarding Animals,E&S,E&S,DOGS/PETS,2015,1
3,Light Pollution,Dick Wavrik,CSC,OUTDOOR LIGHTS,2015,1
4,SHHA HOA Complaince,SHHA,SHHA,SHHA,2015,1
...,...,...,...,...,...,...
806,How'd That Get There? Part 2,Stan Davis,C&P,GEOLOGY,2024,10
807,Progress Being Made: Speed Enforcement Camera ...,Claudia Mitchell,E&S,SAFETY/SECURITY,2024,10
808,"Message from the President, October 2024",Jim Stewart,SHHA,SHHA,2024,10
809,SHHA Bylaws Under Review,Elizabeth Edgren,SHHA,SHHA,2024,10


## View mismatched topics

Check incorrect spaces, punctuation, or just missing Main Topics entries.

In [13]:
# Topic mismatch

topic_list = list(df_histogram['Main Topics'])
print('These topics are in the index but not in the Main Topics list :')
[f for f in set(list(df_index['TOPIC'])) if f not in topic_list]

These topics are in the index but not in the Main Topics list:


['SOLAR ',
 'FIRE (GOATS)',
 'SHHA ',
 'RVs/BOATS',
 'ANN MTG / REPORTS',
 'COVENANTS GENERAL',
 'PROPERTY  MAINT',
 'CS&M',
 'C&P']

## Spellcheck and capitalization correction on article names

In [14]:
list(df_index['Title'])

0                      SHHA Annual Meeting announcements
1                                     Covenant Violation
2         Bernalilloo County Ordinance Regarding Animals
3                                        Light Pollution
4                                    SHHA HOA Complaince
                             ...                        
806                         How'd That Get There? Part 2
807    Progress Being Made: Speed Enforcement Camera ...
808             Message from the President, October 2024
809                             SHHA Bylaws Under Review
810      Let's Protect Our Momma Bear and her three Cubs
Name: Title, Length: 809, dtype: object

In [39]:
# Function to correct spelling and capitalization using OpenAI
def fix_spelling_batch(titles, client):
    titles_text = '\n'.join([f"{i+1}. {title}" for i, title in enumerate(titles)])

    response = client.chat.completions.create(
        model="gpt-4o-mini",  # Specify the GPT-4o-mini model
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"Correct the spelling and capitalization of the titles below. \
            Reply with just the list as it appears, without any additional text.\n\n{titles_text}"}
        ]
        )
    corrected_text = response.choices[0].message.content.strip()
    
    # Split the corrected text back into individual titles
    corrected_titles = corrected_text.split('\n')
    
    # Remove the numbering and strip extra spaces from the corrected titles
    corrected_titles = [title.split('. ', 1)[-1].strip() for title in corrected_titles]
    return corrected_titles

In [45]:
# Split the titles into batches
batch_size = 80
corrected_titles = []

for i in tqdm(range(0, len(df_index), batch_size)):
    # Get the batch of titles
    batch = df_index['Title'].iloc[i:i+batch_size].tolist()
    
    # Correct the batch of titles
    corrected_batch = fix_spelling_batch(batch,client)
    
    # Append the corrected titles to the main list
    corrected_titles.extend(corrected_batch)

# Add the corrected titles to the dataframe
df_index['Corrected_Title'] = corrected_titles

100%|███████████████████████████████████████████| 11/11 [01:50<00:00, 10.01s/it]


In [49]:
# Display the corrected titles
df_index[['Title', 'Corrected_Title']]

,Title,Corrected_Title
0,SHHA Annual Meeting announcements,SHHA Annual Meeting Announcements
1,Covenant Violation,Covenant Violation
2,Bernalilloo County Ordinance Regarding Animals,Bernalillo County Ordinance Regarding Animals
3,Light Pollution,Light Pollution
4,SHHA HOA Complaince,SHHA HOA Compliance
...,...,...
806,How'd That Get There? Part 2,How'd That Get There? Part 2
807,Progress Being Made: Speed Enforcement Camera ...,Progress Being Made: Speed Enforcement Camera ...
808,"Message from the President, October 2024","Message from the President, October 2024"
809,SHHA Bylaws Under Review,SHHA Bylaws Under Review


In [57]:
# View every row that was changed

i_num = 0
for i,row in df_index.iterrows():
    if row['Title'] != row['Corrected_Title']:
        print(row['Title']+' | '+row['Corrected_Title'])
        i_num+=1
print('\n\n There were '+str(i_num)+' titles changed in total. ')

SHHA Annual Meeting announcements | SHHA Annual Meeting Announcements
Bernalilloo County Ordinance Regarding Animals | Bernalillo County Ordinance Regarding Animals
SHHA HOA Complaince | SHHA HOA Compliance
SHHA Annual Meeting announcements | SHHA Annual Meeting Announcements
Weed and Fire Season is Approaching! | Weed and Fire Season Is Approaching!
Birds of Special Interest and Concern are Found in Sandia Heights | Birds of Special Interest and Concern Are Found in Sandia Heights
Parks and Safety   | Parks and Safety
Bear Season is Here | Bear Season Is Here
Street Right of Way Maintenance | Street Right-of-Way Maintenance
Zoning Variance Request made for a Domino's restaurant | Zoning Variance Request Made for a Domino's Restaurant
Surveys and Building Permints - a Refresher | Surveys and Building Permits - A Refresher
Don’t Feed The Bears | Don’t Feed the Bears
SHHA Homeowner Members can Log onto the SHHA Website | SHHA Homeowner Members Can Log Onto the SHHA Website
Sandia Heights

In [53]:
# Save to CSV

df_index[['Title', 'Corrected_Title']].to_csv('title_corrections.csv')

In [54]:
df_index.columns

Index(['Title', 'First Author', '~Relevant Committee', 'TOPIC', 'Year',
       'Month', 'Corrected_Title'],
      dtype='object')

In [60]:
# Update the original xlsx with the corrected titles

df_excel = pd.read_excel(excel_file, sheet_name='INDEX')
df_excel['Titles'] = df_index['Corrected_Title']
df_excel.to_excel(excel_file, sheet_name='INDEX', index=False)

## View unique author names

For spelling check.

In [62]:
set(list(df_index['First Author']))

{'ACC',
 'Alex Deckard',
 'Angela Moreno',
 'Ann B. List',
 'Anne Manning',
 'Anonymous',
 'Arthur Romero',
 'Ben Abruzzo',
 'BernCo',
 'Bob Bower',
 'Bob Thomas',
 "Burt O'Neil",
 'C&P',
 'CS&M',
 'CSC',
 'Cathleen McCaughey',
 'Cheryl Iverson',
 'Cheryl Wieker',
 'Cindi F.',
 'Cindy Mottle',
 'City of Albuquerque',
 'Claudia Mitchell',
 'Colleen Langan-McRoberts',
 'Dale Arendt',
 'Dan Benton',
 'Dan Caudillo',
 'Dave Mottle',
 'David Crossley',
 'David Wilson',
 'Desert Oasis',
 'Diane Barney',
 'Dick Wavrick',
 'Dick Wavrik',
 'E&S',
 'Editor',
 'Eileen Karas',
 'Elaine D. Briseno',
 'Elizabeth Edgren',
 'Emily Rudin',
 'Eric Haskins',
 'Erika Roesler',
 'Franklin S.',
 'George Connor',
 'George Richmond',
 'Haruki Murakami',
 'Hugh Prather',
 'Jan Randall',
 'Janice Strand',
 'Jean Hefflin',
 'Jean Heflin (1988)',
 'Jean Heflin (1991)',
 'Jean Heflin (1993)',
 'Jean Helfin',
 'Jim Stewart',
 'Jo Ann Hamlin',
 'Joe Boyce',
 'Joe Pappe',
 'Jonathan Sutin',
 'Josephine Porter',
 'Jud